In [113]:
from pymatgen import MPRester
from pymatgen import Structure
from pymatgen.io.vasp.sets import MPRelaxSet

In [114]:
#Connect to MP database
mpr = MPRester('sha1JueA3CU5QzB8')

In [124]:
#Get primitive cell from MP
primitive_str = mpr.get_structure_by_material_id('mp-765279')

#print primitive cell structure
print(primitive_str)

Full Formula (Li10 Co1 Ni9 O20)
Reduced Formula: Li10CoNi9O20
abc   :   2.910707  11.034500  11.611659
angles: 113.755689  90.451712  97.476886
Sites (40)
  #  SP           a         b         c    coordination_no  forces                                     magmom
---  ----  --------  --------  --------  -----------------  ---------------------------------------  --------
  0  Li    0.010964  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  1  Li    0.788994  0.600135  0.350986                  6  [0.00757494, 0.00753074, -0.0033043]        0.007
  2  Li    0.58599   0.199114  0.451366                  6  [0.02747461, 0.00637089, 0.00351614]        0.004
  3  Li    0.800111  0.600009  0.851906                  6  [-0.02430739, -0.01976367, -0.00860655]     0.008
  4  Li    0.599036  0.19921   0.94942                   6  [-0.00117946, -0.00117858, 0.0191253]       0.003
  5  Li    0.393918  0.800769  0.04929                   6  [-0.01682144, 0

In [125]:
#Create supercell
#Q: how to decide the supercell size?
primitive_str.make_supercell([[4,0,0],[0,1,0],[0,0,1]])
structure = primitive_str

Full Formula (Li40 Co4 Ni36 O80)
Reduced Formula: Li10CoNi9O20
abc   :  11.642826  11.034500  11.611659
angles: 113.755689  90.451712  97.476886
Sites (160)
  #  SP           a         b         c    coordination_no  forces                                     magmom
---  ----  --------  --------  --------  -----------------  ---------------------------------------  --------
  0  Li    0.002741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  1  Li    0.252741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  2  Li    0.502741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  3  Li    0.752741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  4  Li    0.197248  0.600135  0.350986                  6  [0.00757494, 0.00753074, -0.0033043]        0.007
  5  Li    0.447248  0.600135  0.350986                  6  [0.00757494, 

In [130]:
#Create vacancies in lithium layer. Concentrations are 0, 0.25, 0.5, 0.6, 0.75, 1
#This arrangement may be suitable for all layered-structure lithium cathodes.
#Q: we should hardcode vacancies arrangement for all kinds of materials?
import numpy as np
"""This function hardcode the vacancy position of different concentration."""
def vacancy_position(concentration, structure):
    if concentration == 0.75:
        indice_list = np.arange(0,19,2)
        indice_list = sorted(indice_list,reverse=True)
        return indice_list
    elif concentration == 0.5:
        indice_list = np.arange(0,20)
        indice_list = sorted(indice_list,reverse=True)
        return indice_list
    elif concentration == 0.25:
        indice_list1 = [21,23,24,26,29,31,33,35,37,39]
        indice_list2 = np.arange(0,20)
        indice_list = indice_list1 + indice_list2
        indice_list = sorted(indice_list,reverse=True)
        return indice_list
    elif concentration == 0.6:
        indice_list = np.arange(24,40)
        indice_list = sorted(indice_list,reverse=True)
        return indice_list
    elif concentration == 0:
        indice = np.arange(0,40)
        indice_list = sorted(indice_list,reverse=True)
        return indice_list

"""This function remove the vacancies according to indice list, meanwhile record the coordinates.
   Return a dictionary containing all coordinates."""
def create_vacancy(vac_indice_list, structure):
    vacancy_frac_coords_list =[]
    for i in vac_indice_list:
        site = structure.pop(i)
        vacancy_list_frac_coords.append(site.frac_coords)
    return vacancy_frac_coords_list        

In [133]:
"""This function swap the species at two given indice."""
def swap_element(indice1, indice2, structure):
    specie1 = structure[indice1].specie
    specie2 = structure[indice2].specie
    structure.replace(indice2,specie1)
    structure.replace(indice1,specie2)
    return structure

"""Get the distance between TM atom and all Li atoms. Returnt the indice of the three nearest ones."""
def find_nearest(indice_TM, structure):
    distance = [];
    indice_Li = structure.indices_from_symbol('Li')
    for i in indice_Li:
        d = structure.get_distance(indice_TM,i)
        distance.append(d)
    dictionary = dict(zip(indice_Li,distance))
    dictionary = sorted(dictionary.items(), key = lambda item:item[1])
    three_nearest_Li = [dictionary[0][0],dictionary[1][0],dictionary[2][0]]
    return three_nearest_Li

"""This function swap TM atom and Li atom.
   Then generate the input files.
   Attention: specie should be a string or character with '' !"""
from random import choice
def swap_Li(specie, structure):
    indice_list_specie = structure.indices_from_symbol(specie)
    indice_specie = random.choice(indice_list_specie)
    indice_Li_list = find_nearest(indice_specie, structure)
    for i in indice_Li_list:
        new_structure = structure.copy()
        new_structure = swap_element(indice_specie, i, new_structure)
        file_input = MPRelaxSet(new_structure)
        TM_string = str(indice_specie)
        Li_string = str(i)
        file_input.write_input(specie+TM_string+'Li'+Li_string)
        
"""This function find the three nearest vacancies around the TM atom."""
import math
import numpy as np
def find_nearest_vac(indice_TM, structure, vacancy_frac_coords_list):
    vacancy_distance = []
    index = np.arange(0,len(vacancy_frac_coords_list))
    site_TM = structure[indice_TM]
    xcoord = site_TM.frac_coords[0]
    ycoord = site_TM.frac_coords[1]
    zcoord = site_TM.frac_coords[2]
    for i in range(0,len(vacancy_frac_coords_list)):
        vx = vacancy_frac_coords_list[i][0]
        vy = vacancy_frac_coords_list[i][1]
        vz = vacancy_frac_coords_list[i][2]
        distance_square = math.pow((xcoord-vx),2)+math.pow((ycoord-vy),2)+math.pow((zcoord-vz),2)
        vacancy_distance.append(distance_square)
    vac_dict = dict(zip(index,vacancy_distance))
    vac_dict = sorted(vac_dict.items(), key = lambda item:item[1])
    three_index = [vac_dict[0][0],vac_dict[1][0],vac_dict[2][0]]
    three_nearest_vac = [vacancy_frac_coords_list[three_index[0]],vacancy_frac_coords_list[three_index[1]],vacancy_frac_coords_list[three_index[2]]]
    return three_nearest_vac

"""This function swap TM atom and vacancy position.
   Then generate input files."""
def swap_vac(specie, Li_concentration, structure):
    vac_all_indice = vacancy_position(Li_concentration, structure)
    vac_position = create_vacancy(vac_all_indice,structure)
    indice_list_specie = structure.indices_from_symbol(specie)
    indice_specie = random.choice(indice_list_specie)
    vac_coord_list = find_nearest_vac(indice_specie, structure, vac_position)
    print(indice_vac)
    for i in range(0,len(vac_coord_list)):
        new_structure = structure.copy()
        new_structure.pop(indice_specie)
        new_structure.append(specie, vac_coord_list[i])
        file_input = MPRelaxSet(new_structure)
        TM_string = str(indice_specie)
        vac_string = str(i)
        file_input.write_input(specie+TM_string+'Vac'+vac_string)
    

In [86]:
#Test vacancy_position
vacancy_list = vacancy_position(0.75,structure)
print(vacancy_list)

[18, 16, 14, 12, 10, 8, 6, 4, 2, 0]


In [87]:
#Test create_vacancy(indice_list, structure)
vac_coordinates =create_vacancy(vacancy_list,structure)
print(vac_coordinates)

[array([ 0.64975904,  0.19920965,  0.94941955]), array([ 0.14975904,  0.19920965,  0.94941955]), array([ 0.70002763,  0.60000881,  0.85190555]), array([ 0.20002764,  0.60000881,  0.85190555]), array([ 0.64649758,  0.19911355,  0.45136576]), array([ 0.14649758,  0.19911355,  0.45136576]), array([ 0.69724844,  0.60013536,  0.35098551]), array([ 0.19724844,  0.60013536,  0.35098551]), array([ 0.5027409 ,  0.998655  ,  0.74987648]), array([ 0.0027409 ,  0.998655  ,  0.74987648])]


In [46]:
#Test find_nearest(indice_TM, structure)
three = find_nearest(40, structure)

In [47]:
print(three)

[6, 28, 20]


In [57]:
#Test swap_Li(specie,structure)
import random
from random import choice
swap_Li('Ni',structure)

Full Formula (Li30 Co4 Ni36 O80)
Reduced Formula: Li15Co2(Ni9O20)2
abc   :  11.642826  11.034500  11.611659
angles: 113.755689  90.451712  97.476886
Sites (150)
  #  SP           a         b         c    coordination_no  forces                                     magmom
---  ----  --------  --------  --------  -----------------  ---------------------------------------  --------
  0  Li    0.252741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  1  Li    0.752741  0.998655  0.749876                  6  [-0.06808446, -0.01358486, 0.01107342]      0.006
  2  Li    0.447248  0.600135  0.350986                  6  [0.00757494, 0.00753074, -0.0033043]        0.007
  3  Li    0.947248  0.600135  0.350986                  6  [0.00757494, 0.00753074, -0.0033043]        0.007
  4  Li    0.396498  0.199114  0.451366                  6  [0.02747461, 0.00637089, 0.00351614]        0.004
  5  Li    0.896498  0.199114  0.451366                  6  [0.027474

In [111]:
#Test find_nearest_vac(indice_TM, structure, vacancy_list)
three = find_nearest_vac(40,structure,vac_coordinates)

[array([ 0.64975904,  0.19920965,  0.94941955]), array([ 0.14975904,  0.19920965,  0.94941955]), array([ 0.70002763,  0.60000881,  0.85190555]), array([ 0.20002764,  0.60000881,  0.85190555]), array([ 0.64649758,  0.19911355,  0.45136576]), array([ 0.14649758,  0.19911355,  0.45136576]), array([ 0.69724844,  0.60013536,  0.35098551]), array([ 0.19724844,  0.60013536,  0.35098551]), array([ 0.5027409 ,  0.998655  ,  0.74987648]), array([ 0.0027409 ,  0.998655  ,  0.74987648])]
0.54987586 0.4005939 0.89852187
0.05312283962435732
0.20323965962435728
0.06448494315630113
0.16433316815630103
0.2498786744391043
0.4032569569391041
0.36133153637997456
0.46395895887997446
0.3819942355199889
0.6791291930199888
[(0, 0.05312283962435732), (2, 0.06448494315630113), (3, 0.16433316815630103), (1, 0.20323965962435728), (4, 0.2498786744391043), (6, 0.36133153637997456), (8, 0.3819942355199889), (5, 0.4032569569391041), (7, 0.46395895887997446), (9, 0.6791291930199888)]


In [112]:
three

[array([ 0.64975904,  0.19920965,  0.94941955]),
 array([ 0.70002763,  0.60000881,  0.85190555]),
 array([ 0.20002764,  0.60000881,  0.85190555])]

In [128]:
#Test swap_vac(specie,Li_concentration,structure)
swap_vac('Ni',0.75,structure)

[array([ 0.64975904,  0.19920965,  0.94941955]), array([ 0.14975904,  0.19920965,  0.94941955]), array([ 0.70002763,  0.60000881,  0.85190555]), array([ 0.20002764,  0.60000881,  0.85190555]), array([ 0.64649758,  0.19911355,  0.45136576]), array([ 0.14649758,  0.19911355,  0.45136576]), array([ 0.69724844,  0.60013536,  0.35098551]), array([ 0.19724844,  0.60013536,  0.35098551]), array([ 0.5027409 ,  0.998655  ,  0.74987648]), array([ 0.0027409 ,  0.998655  ,  0.74987648])]
0.7499487075 0.00069284 0.99961992
0.05196697047447354
0.4021566379744735
0.38349128048108794
0.6834123529810878
0.3506555384124864
0.7041066684124861
0.7828352510780711
1.085535521078071
1.119411957467817
1.6166197624678167
[(0, 0.05196697047447354), (4, 0.3506555384124864), (2, 0.38349128048108794), (1, 0.4021566379744735), (3, 0.6834123529810878), (5, 0.7041066684124861), (6, 0.7828352510780711), (7, 1.085535521078071), (8, 1.119411957467817), (9, 1.6166197624678167)]
[array([ 0.64975904,  0.19920965,  0.949419